In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import collections

sys.path.append("../src/")
sys.path.append("../src/utils")

In [2]:
%load_ext autoreload
%autoreload 1
%aimport datasets

In [3]:
from general import *

In [4]:
discord_path = '../data/social/discord/'
facebook_paths = ['../data/social/facebook1/',
                  '../data/social/facebook2/']
instagram_path = ['../data/social/instagram/']

In [5]:
def load_all(loader, dir):
    buff = []
    if not os.path.isdir(dir): return buff
    for i in os.listdir(dir):
        buff.append(loader(dir + i))
    return buff
        

chats = []
chats += load_all(datasets.load_discord, discord_path)

for fb in facebook_paths:
    chats += load_all(datasets.load_messenger, fb + 'messages/inbox/')
    chats += load_all(datasets.load_messenger, fb + 'messages/archived_threads/')
    chats += load_all(datasets.load_messenger, fb + 'messages/e2ee_cutover/')
    chats += load_all(datasets.load_messenger, fb + 'messages/filtered_threads/')

original_chats = chats

In [6]:
len(original_chats)

171

In [7]:
datasets.view_excerpt(original_chats[9])

patztablook22,lol
jasaf3,My life and relation to reality is becoming exceedingly odd
jasaf3,"Mental asylums seem like liminal spaces, however"
jasaf3,They really want to keep the capacity at the minimum
jasaf3,So it's not a worthwhile cope
jasaf3,"Do you think I have narcissism or asperger's syndrome? If not, do I have any other mental illness? If so, which one?"
jasaf3,"Then again, since the Colin DeYoung shift, there may not even be mental illnesses, he wants to fuse together his Cybernetic Big Five model with the next release of the DSM."
jasaf3,what a fucking chad
jasaf3,Hmm
jasaf3,How come you are so focused and work so hard in spite of your low conscientiousness?


In [8]:
def info(chats):
    all = [m for chat in chats for m in chat]
    sample = np.random.choice(all, size=1000)
    
    lengths = [len(m.body) for m in all if len(m.body) != 0]
    languages = collections.Counter(filter(lambda l: l is not None, datasets.get_languages(sample, languages=['en', 'cs', 'sk', 'ru', 'de'], threshold=0)))
    users = collections.Counter(map(lambda m: m.user, all))
    
    print("Total messages  ", len(all))
    print("Length/message  ", f"mean: {np.mean(lengths):.1f}, quantiles 10-50-90: {np.quantile(lengths, 0.10)} {np.quantile(lengths, 0.50)} {np.quantile(lengths, 0.95)}")
    print("With attachment ", f"{len([m for m in all if m.attachments]) / len(all) * 100:.1f}%")
    print("With reaction   ", f"{len([m for m in all if m.reactions]) / len(all) * 100:.1f}%")
    print("Languages       ", ', '.join([f"{l}: {round(f/languages.total()*100)}%" for l,f in languages.most_common()]))
    print("Users           ", ', '.join([f"{u}: {round(f/users.total()*100)}%" for u,f in users.most_common(8)]))

info(original_chats)

Total messages   516948
Length/message   mean: 33.1, quantiles 10-50-90: 3.0 21.0 93.0
With attachment  6.3%
With reaction    14.2%
Languages        en: 63%, cs: 27%, sk: 6%, de: 4%, ru: 0%
Users            Patztablook TwentyTwo: 43%, Scanthning Brot: 14%, Marie Holá: 13%, jasaf3: 9%, patztablook22: 6%, Drew Mac: 3%, Jan Zasadil: 2%, : 2%


In [9]:
agents = [{'Patztablook TwentyTwo', 'patztablook22', 'patrik zavoral', 'patz'},
          {'Scanthning Brot', 'jasaf3'},
          {'Drew Mac'}]

additional = ['Mank', 'Mari', 'Jáš', 'Sběratel']

def rename(chats):
    all = [m for chat in chats for m in chat]
    users = datasets.get_users(all) + additional

    result = []
    for chat in chats:
        users = datasets.get_users(chat)
        for aa in agents:
            relevant = len(aa.intersection(users)) > 0
            if not relevant: continue
            mapper = datasets.make_name_mapper(users, fixed={agent: 'p' for agent in aa})
            result.append([datasets.change_names(m, mapper) for m in chat])
            print('  ', 'users ', users)
            print('  ', 'agents', aa)
            print('=>', 'users ', datasets.get_users(result[-1]))
            print()
            
    return result

renamed_chats = rename(original_chats)

   users  ['patztablook22', 'sberatelbananu']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Patztablook TwentyTwo'}
=> users  ['p', 'shannon']

   users  ['burnytech', 'patztablook22']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Patztablook TwentyTwo'}
=> users  ['angela', 'p']

   users  ['tlapik', 'patztablook22']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Patztablook TwentyTwo'}
=> users  ['p', 'darrell']

   users  ['patztablook22', 'kotlin_enjoyer']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Patztablook TwentyTwo'}
=> users  ['p', 'francisco']

   users  ['tamkish', 'patztablook22']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Patztablook TwentyTwo'}
=> users  ['p', 'david']

   users  ['patztablook22', 'yuk1puk3z']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Patztablook TwentyTwo'}
=> users  ['andrea', 'p']

   users  ['jasaf3', 'patztablook22', 'yuk1puk3z']
   agents {'patz', 'patztablook22', 'patrik zavoral', 'Pat

In [10]:
def censor(message):
    def c(tag, pattern, replacer=None):
        nonlocal message
        message = datasets.censor(message, pattern, replacer=None, tag=tag)

    def blacklist(pattern):
        nonlocal message
        message = datasets.censor(message, fr'(?s).*{pattern}.*', replacer=None, tag=pattern)

    c('email', r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    c('url',   r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b|ftp:\/\/[A-Za-z0-9._%+-]+:[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    c('url',   r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&#+~]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    c('bank',  r'(\d{1,6}-)?\d{1,10}/\d{4}')
    c('phone', r'\+?\d[\d -]{8,}\d')
    c('address', ['Třešňová', '914'])
    c('birthday', ['23 dec', 'dec 23', 'december 23', '23 december', '23.', '23/'])

    blacklist('satisfied with your genes')
    blacklist('satisfied with ur genes')
    blacklist('se svymi geny')
    blacklist('se svými geny')
    blacklist('popsal by ses')
    blacklist('would you describe urself')
    blacklist('left the group.')
    blacklist('changed the group photo.')
    blacklist('created the group.')
    blacklist('started a call')
    blacklist('the call ended')
    blacklist('joined the call')
    
    return message

censored_chats = llmap(censor, renamed_chats[:])


KeyboardInterrupt



In [ ]:
i = range(len(censored_chats))
i = 3
if isinstance(i, int): i = [i]
for j in i:
    datasets.view_excerpt(censored_chats[j], 
                          # where=lambda m: 'would you describe urself' in m.body
                          where=lambda m: m.issues
                          # where=lambda m: 'group' in m.body
                         )

In [ ]:
def censorship_info(chats):
    all = [m for chat in chats for m in chat]
    censorships = []
    for m in all:
        for issue in m.issues:
            if issue.type == 'censor': censorships.append(issue.data['tag'])

    counter = collections.Counter(censorships)
    print("Censorship rate:\n" + f"{round(len(censorships) / len(all), 3):.3f}")
    print()
    print("Censorship tags:\n" + ', '.join([f"{tag}: {round(100*count/counter.total(), 2)}%" for tag, count in counter.most_common(20)]))

censorship_info(censored_chats)

In [39]:
agents = ['p']

def enhance_actions(a):
    a = datasets.add_control_actions(a, agents=agents, duration_limit=300, pause_limit=90, count_limit=7, idle_rate=1)
    a = datasets.mask_actions(a, lambda i: i.user not in agents)
    a = datasets.mask_actions(a, lambda i: i.type == 'attachment')
    return a

actions = lmap(datasets.chat_to_actions, censored_chats)
actions = lmap(enhance_actions, actions)

eta 10 s
eta 58 s
eta 28 s
eta 14 s


In [42]:
def info(actions):
    agents = ['p']
    all = [a for aa in actions for a in aa]
    agent = lfilter(lambda a: a.user in agents, all)
    msg = lfilter(lambda a: a.type == 'message', agent)
    react = lfilter(lambda a: a.type == 'reaction', agent)
    sample = np.random.choice(msg, size=1000)
    
    lengths = [len(m.data['body']) for m in msg if len(m.data['body']) != 0]
    languages = collections.Counter(datasets.get_languages(datasets.actions_to_chat(sample), languages=['en', 'cs', 'sk', 'ru', 'de'], threshold=0))
    agent_types = collections.Counter([a.type for a in agent])
    
    print("Total actions       ", len(all))
    print("Agent actions       ", len(agent))
    print()
    print("Agent action types  ", ', '.join([f"{t}: {round(100 * count/agent_types.total())}%" for t, count in agent_types.most_common(len(agent_types))]))
    print()
    print("Agent messages details:")
    print("  Length/message    ", f"mean: {np.mean(lengths):.1f}, quantiles 10-50-90: {np.quantile(lengths, 0.10)} {np.quantile(lengths, 0.50)} {np.quantile(lengths, 0.95)}")
    print("  Languages         ", ', '.join([f"{l}: {round(f/languages.total()*100)}%" for l,f in languages.most_common()]))
    # print("Users           ", ', '.join([f"{u}: {round(f*100)}%" for u,f in sorted(users.items(), key=lambda i: -i[1])]))
    

info(actions)

Total actions        1064757
Agent actions        651609

Agent action types   message: 55%, idle: 34%, reaction: 6%, attachment: 4%

Agent messages details:
  Length/message     mean: 33.9, quantiles 10-50-90: 3.0 21.0 96.0
  Languages          en: 71%, cs: 16%, None: 7%, sk: 4%, de: 2%, ru: 0%


In [89]:
i = np.random.choice(len(actions))
a = actions[i]
j = np.random.choice(len(a))
a = a[j:j+45]
# datasets.simulate(a, sleep=0.1)

for i in a:
    s, m = datasets.action_to_string(i, return_mask=True)
    datasets.view_masked(s, m)

In [90]:
actions_path = '../data/actions.pkl'
# pickle good enough, only around 50 MB

import pickle
with open(actions_path, 'wb') as f:
    pickle.dump(actions, f)